In [111]:
import os
from tqdm import tqdm
import requests, json, pandas as pd, numpy as np
from requests.models import HTTPError
from datetime import datetime, timedelta

In [118]:
'''
Merge all the Driver Response Responses into one csv
'''

arr = []

for _ in range(1, 9):
    if _ == 1:
        arr.append(pd.read_csv('Driver_Responses/Response_' + str(_) + '.csv'))
    else:
        arr.append(pd.read_csv('Driver_Responses/Response_' + str(_) + '.csv'))

arr.append(pd.read_csv(r'Driver_Responses/Response_Missing.csv'))

df = pd.concat(arr)

df.to_csv('Data/Race_History_Table.csv')

# Delete Dataframe
del df

In [119]:
# Load Driver Table
df_drivers = pd.read_csv(r'Data/Driver_Table.csv')
drivers = df_drivers.loc[:, 'driver_id']

# Load 1st place Table
df_first = pd.read_csv(r'Data/First_Table.csv')

# Load Race History Table
hist_table = pd.read_csv(r'Data/Race_History_Table.csv')

hist_table.head()

,Unnamed: 0,season,round,circuit_id,status,time,position,points,driver_id,team_id,date
0,0,1962,4,essarts,Withdrew,NaN,21,0.0,abate,lotus-climax,1962-07-08
1,1,1963,7,monza,Withdrew,NaN,32,0.0,abate,porsche,1963-09-08
2,2,1951,1,bremgarten,Magneto,NaN,17,0.0,abecassis,hwm,1951-05-27
3,3,1952,1,bremgarten,Halfshaft,NaN,17,0.0,abecassis,hwm,1952-05-18
4,4,1983,9,silverstone,Did not qualify,NaN,29,0.0,acheson,ram,1983-07-16


In [120]:
# Initalize
hist_table['true_time'] = ""

# Get Drivers true time for every race
hist_table['true_time'] = hist_table[['driver_id', 'season', 'round', 'time']].apply(lambda x: get_race_time(*x), axis=1)

# Drop Time Column from original csv
hist_table.drop(labels=['time'], axis=1, inplace=True)

# Drops the Unnamed: 0 column that gets generated from Pandas
hist_table.drop(labels=['Unnamed: 0'], axis=1, inplace=True)

hist_table.to_csv('Data/Race_History_Table.csv',index=False)
# del hist_table
# del df_drivers
# del df_first

In [123]:
# Sort by Season, Round so most current race is on top
hist_table = hist_table.sort_values(['season','round'],ascending=False)

hist_table.head()

,season,round,circuit_id,status,position,points,driver_id,team_id,date,true_time
13120,2021,22,yas_marina,Accident,16,0.0,latifi,williams,2021-12-12,0:00:00
20983,2021,22,yas_marina,+1 Lap,14,0.0,mick_schumacher,haas,2021-12-12,0:00:00
8148,2021,22,yas_marina,Finished,5,10.0,gasly,alphatauri,2021-12-12,1:30:23.876000
21886,2021,22,yas_marina,+1 Lap,13,0.0,stroll,aston_martin,2021-12-12,0:00:00
17025,2021,22,yas_marina,Engine,15,0.0,perez,red_bull,2021-12-12,0:00:00


In [104]:
hist_table.loc[(hist_table['driver_id'] == 'hamilton')]

,season,round,circuit_id,status,position,points,driver_id,team_id,date,true_time
9536,2007,1,albert_park,Finished,3,6.0,hamilton,mclaren,2007-03-18,1:25:47.365000
9537,2007,2,sepang,Finished,2,8.0,hamilton,mclaren,2007-04-08,1:32:32.487000
9538,2007,3,bahrain,Finished,2,8.0,hamilton,mclaren,2007-04-15,1:33:29.875000
9539,2007,4,catalunya,Finished,2,8.0,hamilton,mclaren,2007-05-13,1:31:43.020000
9540,2007,5,monaco,Finished,2,8.0,hamilton,mclaren,2007-05-27,1:40:33.424000
...,...,...,...,...,...,...,...,...,...,...
9819,2021,18,rodriguez,Finished,2,18.0,hamilton,mercedes,2021-11-07,1:38:55.641000
9820,2021,19,interlagos,Finished,1,25.0,hamilton,mercedes,2021-11-14,1:32:22.851000
9821,2021,20,losail,Finished,1,25.0,hamilton,mercedes,2021-11-21,1:24:28.471000
9822,2021,21,jeddah,Finished,1,26.0,hamilton,mercedes,2021-12-05,2:06:15.118000


In [105]:
def convertHours(raw_time):
    ''' Format H:M:S:Micro '''
    dirty_time = datetime.strptime(raw_time,'%I:%M:%S.%f').time()
    clean_time = timedelta(hours=dirty_time.hour, minutes=dirty_time.minute, seconds=dirty_time.second, microseconds=dirty_time.microsecond)
    return clean_time

In [106]:
def convertMinute(raw_time):
    ''' Format M:S:Micro '''
    dirty_time = datetime.strptime(raw_time,'%M:%S.%f').time()
    clean_time = timedelta(minutes=dirty_time.minute, seconds=dirty_time.second, microseconds=dirty_time.microsecond)
    return clean_time

In [107]:
def convertSeconds(raw_time):
    ''' Format S:Micro '''
    dirty_time = datetime.strptime(raw_time, '%S.%f').time()
    clean_time = timedelta(seconds=dirty_time.second, microseconds=dirty_time.microsecond)
    return clean_time

In [108]:
def get_race_time(driver_id, season, rd, time) -> any:
    '''
    Convert Drivers time, either winning time or + time to complete track time
    API has alot of dirty data so alot of cleaning is required
    '''

    # Time doesn't Exist
    # Ex: +1 Lap, Engine, Throttle etc...
    # For now make it time of 0.0
    if type(time) == float:
        clean_time = timedelta(seconds=0, microseconds=0)
        #print(driver_id,season,rd,clean_time)
        return str(clean_time)

    # Gets Race winner
    winner_frame = (df_first.loc[(df_first['season'] == season) & (df_first['round'] == rd)])[['time', 'driver_id', 'season', 'round']]

    if winner_frame.iloc[0,0].count(':') == 2:
        # Rare case hours if 0 in the time then removes the 0 in the hour slot
        try:
            clean_winner_time = convertHours(winner_frame.iloc[0,0])
        
        except ValueError:
            bad_time = winner_frame.iloc[0,0]
            better_time = bad_time[2:]
            clean_winner_time = convertMinute(better_time)

    elif winner_frame.iloc[0,0].count(':') == 1:
        clean_winner_time = convertMinute(winner_frame.iloc[0,0])
    
    else:
        clean_winner_time = convertSeconds(winner_frame.iloc[0,0])

    if winner_frame.iloc[0,1] == driver_id:
        return str(clean_winner_time)

    # Removes + in front of the times
    elif '+' in time:

        dirty_time = time[1:]

        # Rare case 's' or 'sec' is in the time
        # or
        # Rare case there is a space infront of the hour time EX: ' 1.06.7'
        if 'sec' in dirty_time:
            dirty_time = dirty_time[:-4]

        elif 's' in dirty_time:
            dirty_time = dirty_time[:-1]

        elif dirty_time[:1] == ' ':
            dirty_time = dirty_time[1:]

        # Formatting of the time
        # Rare case where There is a time with no seconds; adds .0 seconds to time EX: '1:10'
        if ':' in dirty_time:
            try:
                # min, sec, micro
                clean_time = convertMinute(dirty_time)
            except ValueError:
                
                dirty_time += '.0'
                clean_time = convertMinute(dirty_time)

        else:
            try:
                # sec, micro
                clean_time = convertSeconds(dirty_time)

            except ValueError:
                # Rare case Min is => 60
                #print(dirty_time)

                # Separate Seconds and Millseconds
                sec, mill = map(int,dirty_time.split('.'))

                # Get decimal value of Minute and second
                convert = str(round(sec/60,1))

                # Separate Minutes and Seconds
                min, sec = map(int, convert.split('.'))

                # Format into readable time
                new_time = f'{min}:{sec}.{mill}'
                clean_time = convertMinute(new_time)


        # Combine Winner time with driver time to get total lap time
        total_time = str(clean_winner_time + clean_time)
        return total_time